In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('DC_Channel.xlsx')
data.dropna(axis=0, how='any', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224984 entries, 0 to 227627
Data columns (total 3 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   attribution_technical  224984 non-null  object        
 1   attribution_survey     224984 non-null  object        
 2   account_creation_date  224984 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 6.9+ MB


In [3]:
data['account_creation_date'] = data['account_creation_date'].dt.date
data['account_creation_date'] = (pd.DatetimeIndex(data['account_creation_date']).year)*100 + (pd.DatetimeIndex(data['account_creation_date']).month)
data

,attribution_technical,attribution_survey,account_creation_date
0,facebook,facebook,202001
1,facebook,facebook,202003
2,organic,facebook,201912
3,search,tv,202001
4,discovery,youtube,201910
...,...,...,...
227623,facebook,facebook_organic,201911
227624,google_organic,referral,201912
227625,facebook,facebook,201912
227626,facebook,facebook,202001


In [4]:
#Channel Spend
channel_spend = pd.read_excel('5-Channel-Spend.xlsx',index_col=0)
channel_spend

,bing,display,facebook,search,youtube
Date,,,,,
201907,400,12,9000,13000,90
201908,900,20,13000,18500,180
201909,1000,13,10500,19000,100
201910,1100,19,17000,24000,130
201911,1300,29,23000,25000,550
201912,300,31,16000,38000,900
202001,2100,148,11000,41000,4360
202002,3700,94,14000,44000,2420


In [24]:
data_t = data[data['account_creation_date']==201907].reset_index(drop=True)

In [6]:
data_t.loc[0]

attribution_technical         organic
attribution_survey       public_radio
account_creation_date          201907
Name: 0, dtype: object

In [7]:
Channel_list = ['bing', 'display', 'facebook', 'search', 'youtube']
Channel_list.index('youtube')
Channel_list[4]

'youtube'

In [16]:
# give weight to survey&technical
Channel_list = ['bing', 'display', 'facebook', 'search', 'youtube']
time_list = [201907,201908,201909,201910,201911,201912,202001,202002]

# Create empty dataframe
result = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)

In [17]:
#count the numeber of people coverted.
for t in time_list:
    count_list = [0,0,0,0,0]
    data_t = data[data['account_creation_date']==t].reset_index(drop=True)
    for i in range(0,len(data_t)):
        a = data_t['attribution_survey'][i] 
        b = data_t['attribution_technical'][i]
        if a in Channel_list and b in Channel_list:
            count_list[Channel_list.index(a)]+= 0.5
            count_list[Channel_list.index(b)]+= 0.5
        elif a in Channel_list and b not in Channel_list:
            count_list[Channel_list.index(a)]+= 1
        elif a not in Channel_list and b in Channel_list:
            count_list[Channel_list.index(b)]+= 1
        else:
            continue
    result.loc[t]= count_list
        
result

,bing,display,facebook,search,youtube
201907,89,32,16028,1378.5,75.5
201908,54,48.5,16429,1953,250.5
201909,62.5,9,12309.5,1876,199
201910,63.5,38.5,14484.5,2204.5,263
201911,70.5,64.5,18510.5,2348.5,295
201912,23.5,83.5,14434,3497,237
202001,136,386.5,10706,3929,741.5
202002,230,243.5,12977.5,4294,528


In [18]:
spend_list = []
for c in Channel_list:
    spend_list.append(channel_spend[c].sum())

spend_list

[10800, 366, 113500, 222500, 8730]

In [19]:
convert_list = []
for c in result:
    convert_list.append(result[c].sum())

convert_list

[729.0, 906.0, 115879.0, 21480.5, 2589.5]

In [20]:
avg_cac_list = []
for i in range(0,5):
    if convert_list[i] == 0:
        avg_cac_list.append(0)
    else:
        avg_cac = round(spend_list[i]/convert_list[i],2)
        avg_cac_list.append(avg_cac)

avg_cac_list

[14.81, 0.4, 0.98, 10.36, 3.37]

In [21]:
# Create empty dataframe
marg_cac = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)
for c in Channel_list:
    marg_cac_list = []
    for t in time_list:
        if result[c][t] == 0:
            marg_cac_list.append(0)
        else:
            marg_cac_value = round(channel_spend[c][t]/result[c][t],2)
            marg_cac_list.append(marg_cac_value)
    marg_cac[c] = marg_cac_list

marg_cac

,bing,display,facebook,search,youtube
201907,4.49,0.38,0.56,9.43,1.19
201908,16.67,0.41,0.79,9.47,0.72
201909,16.00,1.44,0.85,10.13,0.50
201910,17.32,0.49,1.17,10.89,0.49
201911,18.44,0.45,1.24,10.65,1.86
201912,12.77,0.37,1.11,10.87,3.80
202001,15.44,0.38,1.03,10.44,5.88
202002,16.09,0.39,1.08,10.25,4.58


In [25]:
marg_cac.loc['average_CAC'] = [14.81, 0.4, 0.98, 10.36, 3.37]
marg_cac

,bing,display,facebook,search,youtube
201907,4.49,0.38,0.56,9.43,1.19
201908,16.67,0.41,0.79,9.47,0.72
201909,16.00,1.44,0.85,10.13,0.50
201910,17.32,0.49,1.17,10.89,0.49
201911,18.44,0.45,1.24,10.65,1.86
201912,12.77,0.37,1.11,10.87,3.80
202001,15.44,0.38,1.03,10.44,5.88
202002,16.09,0.39,1.08,10.25,4.58
average_CAC,14.81,0.40,0.98,10.36,3.37


In [26]:
marg_cac.to_csv('ca_merged.csv')